In [ ]:
import os
import re
import sys
import pickle
import numpy as np
from numpy.random import RandomState, SeedSequence, MT19937
import matplotlib.pyplot as plt

powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2020 SP4\Python\3.8'
if powerfactory_path not in sys.path:
    sys.path.append(powerfactory_path)
import powerfactory as pf

from pfcommon import *

cmap_name = 'viridis'

In [ ]:
app = pf.GetApplication()
if app is None:
    raise Exception('Cannot get PowerFactory application')
else:
    print('Successfully obtained PowerFactory application.')

In [ ]:
project_name = '\\Terna_Inerzia\\39 Bus New England System'
err = app.ActivateProject(project_name)
if err:
    raise Exception(f'Cannot activate project {project_name}')
print(f'Successfully activated project {project_name}.')

In [ ]:
project = app.GetActiveProject()
if project is None:
    raise Exception('Cannot get active project')
print('Successfully obtained active project.')

In [ ]:
project_folders = {}
for folder_name in ('study',):
    project_folders[folder_name] = app.GetProjectFolder(folder_name)
    if project_folders[folder_name] is None:
        raise Exception(f'No folder "{folder_name}" present')
    print(f'Successfully obtained folder "{folder_name}".')

In [ ]:
generators = app.GetCalcRelevantObjects('*.ElmSym')
lines = app.GetCalcRelevantObjects('*.ElmLne')
buses = app.GetCalcRelevantObjects('*.ElmTerm')
loads = app.GetCalcRelevantObjects('*.ElmLod')
n_generators, n_lines, n_buses, n_loads = len(generators), len(lines), len(buses), len(loads)
print(f'There are {n_generators} generators.')
print(f'There are {n_lines} lines.')
print(f'There are {n_buses} buses.')
print(f'There are {n_loads} loads.')

In [ ]:
for line in lines:
    line_type = line.GetAttribute('typ_id')
    vrating = line_type.GetAttribute('uline')
    if False: print(f'{line.loc_name}: Vrating = {vrating:6.1f} kV.')

In [ ]:
P_gen = {}
S_gen = {}
H_gen = {}

default_H = {
    1:  5.0,
    2:  4.33,
    3:  4.47,
    4:  3.57,
    5:  4.33,
    6:  4.35,
    7:  3.77,
    8:  3.47,
    9:  3.45,
    10: 4.20
}
coeff = 1
H = {k: coeff*v for k,v in default_H.items()}
for generator in generators:
    i = get_ID(generator)
    generator_type = generator.GetAttribute('typ_id')
    generator_type.h = H[i]
    P_gen[i] = generator.GetAttribute('pgini')
    S_gen[i] = generator_type.GetAttribute('sgn')
    H_gen[i] = generator_type.GetAttribute('h')
    print(f'{generator.loc_name}: P = {P_gen[i]:4.0f} MW, S = {S_gen[i]:5.0f} MVA, inertia = {H_gen[i]:5.2f} s.')

In [ ]:
areas_map = {
    1: [2, 3, 10],
    2: [4, 5, 6, 7],
    3: [8, 9],
    4: [1]
}
H_area = {}   # inertia
E_area = {}   # energy
M_area = {}   # momentum
for area_id,generator_ids in areas_map.items():
    num, den = 0,0
    for generator_id in generator_ids:
        num += S_gen[generator_id] * H_gen[generator_id]
        den += S_gen[generator_id]
    H_area[area_id] = num / den 
    E_area[area_id] = num * 1e-3
    M_area[area_id] = 2 * num * 1e-3 / 60
print('Area inertias:  [{}] s.'.format(', '.join(list(map(lambda s: f'{s:5.2f}', H_area.values())))))
print('Area energies:  [{}] GW s.'.format(', '.join(list(map(lambda s: f'{s:5.2f}', E_area.values())))))
print('Area momentums: [{}] GW s^2.'.format(', '.join(list(map(lambda s: f'{s:5.2f}', M_area.values())))))

In [ ]:
P_load = {}
Q_load = {}
for load in loads:
    i = get_ID(load)
    P_load[i] = load.GetAttribute('plini')
    Q_load[i] = load.GetAttribute('qlini')
    print(f'{load.loc_name}: P = {P_load[i]:7.2f} MW, Q = {Q_load[i]:6.2f} MVA')

In [ ]:
Vrating_bus = {}
for bus in buses:
    i = get_ID(bus)
    Vrating_bus[i] = bus.GetAttribute('uknom')
    print(f'{bus.loc_name}: Vrating = {Vrating_bus[i]:7.2f} kV')

## Load flow analysis

In [ ]:
study_case_name = '1. Power Flow'
lf_res = run_load_flow(app, project_folders['study'], generators, loads, buses, study_case_name)
print_load_flow(lf_res)

## Save the data

In [ ]:
data = {
    'P_gen': P_gen, 'S_gen': S_gen, 'H_gen': H_gen,
    'areas_map': areas_map,
    'E_area': E_area, 'M_area': M_area, 'H_area': H_area,
    'P_load': P_load, 'Q_load': Q_load,
    'Vrating_bus': Vrating_bus,
    'PF': lf_res
}
pickle.dump(data, open('IEEE39_load_flow_digsilent.pkl','wb'))